In [1]:

from langchain_community.document_loaders import TextLoader
import os
import pandas as pd
import glob

import glob
import os
import json
import mimetypes
from bs4 import BeautifulSoup

class RecursiveTextLoader:
    def __init__(self, root_dir):
        self.root_dir = root_dir

    def load(self):
        documents = []
        for filepath in glob.glob(os.path.join(self.root_dir, '**'), recursive=True):
            if os.path.isfile(filepath):
                mime_type, _ = mimetypes.guess_type(filepath)
                if mime_type == 'application/json':
                    with open(filepath, 'r', encoding='utf-8') as file:
                        documents.append(json.load(file))
                elif mime_type == 'text/html' or self._looks_like_html(filepath):
                    with open(filepath, 'r', encoding='utf-8') as file:
                        soup = BeautifulSoup(file, 'html.parser')
                        documents.append(soup.get_text())
                elif mime_type == 'text/plain':
                    with open(filepath, 'r', encoding='utf-8') as file:
                        documents.append(file.read())
        return documents

    def _looks_like_html(self, filepath):
        with open(filepath, 'r', encoding='utf-8') as file:
            head = file.read(1024)  # Read the first 1024 bytes
            return '<html' in head or '<!DOCTYPE html>' in head

# Assuming your data folder is at "../data/"
loader = RecursiveTextLoader("../data/")
documents = loader.load()
# Assuming documents is a list of strings or convertible to string
with open('output.txt', 'a', encoding='utf-8') as file:
    for document in documents:
        # Remove all linebreaks from the document and then append
        file.write(str(document).replace('\n', '') + '\n')


In [3]:
def remove_redundancies(input_file, output_file=None):
    """
    Removes consecutive redundant lines from a text file while preserving the order.

    :param input_file: Path to the input text file.
    :param output_file: Path to the output text file. If None, overwrites the input file.
    """
    if output_file is None:
        output_file = input_file

    with open(input_file, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Use a loop to remove consecutive duplicate lines
    unique_lines = []
    previous_line = None
    for line in lines:
        if line != previous_line:
            unique_lines.append(line)
            previous_line = line

    with open(output_file, 'w', encoding='utf-8') as file:
        file.writelines(unique_lines)

# Usage
input_path = 'output.txt'
remove_redundancies(input_path)